In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [13]:
import httpx

from urllib.parse import urljoin

from will_it_saturate.server import (
    FastAPIUvicornServer,
    NginxDockerServer,
    DjangoGunicornWSGIServer,
)
from will_it_saturate.core import FilesystemCreator, Epoch, Result, BenchmarkFile
from will_it_saturate.client import HttpxClient, AioHttpClient, WrkClient
from will_it_saturate.persistence import SqliteRepository

<IPython.core.display.Javascript object>

# Local Benchmark

In [ ]:
# dont_test

byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7
file_creator = FilesystemCreator()


# server = DjangoGunicornWSGIServer()
server = FastAPIUvicornServer()
# client = AioHttpClient(name="aiohttp")
client = HttpxClient(name="httpx")
# repository = SqliteRepository.build_repository("foo.db")

epoch = Epoch(file_size=file_size, duration=duration, file_creator=file_creator)

epoch.create_files()

result = BenchmarkResult(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

if not server.started:
    server.start

result.elapsed = client.measure(epoch)

<IPython.core.display.Javascript object>

# Remote Benchmark

In [3]:
byte = 8
gigabit = 10 ** 9
bandwidth = gigabit / byte
duration = 3
file_size = 10 ** 7
file_creator = FilesystemCreator()

epoch = Epoch(file_size=file_size, duration=duration, file_creator=file_creator)

<IPython.core.display.Javascript object>

In [11]:
control_base = "http://localhost:8001/"
control_epochs = urljoin(control_base, "epochs/")

r = httpx.post(control_epochs, json=epoch.dict_without_file_creator())
print(r.status_code, len(r.json()))

200 6


<IPython.core.display.Javascript object>

In [15]:
r.json()

{'file_size': 10000000,
 'duration': 3,
 'bandwidth': 125000000,
 'files': [{'number': 0,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': '61d3e3ca52e4878ed4b9751d204e64d9',
   'creator': {}},
  {'number': 1,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': 'd1d4eceb00bbfcae752984ad735ef906',
   'creator': {}},
  {'number': 2,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': 'e62db9c0c03f92d446d9d51d6b98db03',
   'creator': {}},
  {'number': 3,
   'base_path': '10000000_3_125000000',
   'size': 10000000,
   'data_root': 'data',
   'hostname': 'localhost',
   'port': 8000,
   'checksum': '259b3debe22dbf7a8252046889bcff4b',
   'creator': {}},
  {'number': 4,
   'base_path': '10000000_3_125000000',
   'si

<IPython.core.display.Javascript object>

In [17]:
for bfile in r.json()["files"]:
    kwargs = {k: v for k, v in bfile.items() if k != "creator"}
    epoch.files.append(BenchmarkFile(**kwargs))

<IPython.core.display.Javascript object>

In [5]:
server = FastAPIUvicornServer()

<IPython.core.display.Javascript object>

In [6]:
server.dict()

{'name': 'fastAPI/uvicorn'}

<IPython.core.display.Javascript object>

In [7]:
server = FastAPIUvicornServer()
control_servers = urljoin(control_base, "servers/")
r = httpx.post(control_servers, json=server.dict())
print(r.status_code, r.json())

200 {'name': 'fastAPI/uvicorn'}


<IPython.core.display.Javascript object>

In [8]:
client = HttpxClient(name="httpx")


result = Result(
    server=server.name,
    client=client.name,
    file_size=epoch.file_size,
    complete_size=epoch.complete_size,
)

<IPython.core.display.Javascript object>

In [18]:
%%time
result.elapsed = client.measure(epoch)

measure
CPU times: user 6.02 ms, sys: 9.19 ms, total: 15.2 ms
Wall time: 8.89 s


<IPython.core.display.Javascript object>

In [19]:
result

Result(server='fastAPI/uvicorn', client='httpx', file_size=10000000, elapsed=6.465179579, complete_size=375000000)

<IPython.core.display.Javascript object>

In [20]:
result.bytes_per_second

58003029.21485176

<IPython.core.display.Javascript object>